In [70]:
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf

from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.io import read_image
import torchvision.models as models
import matplotlib.pyplot as plt
import pathlib

import albumentations as A

import torchvision.transforms as transforms
import os


In [ ]:
!unzip public_tests.zip

Archive:  public_tests.zip
replace 00_test_img_gt/gt.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.mobilenet_v2(pretrained=True)
# model.to(device)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_ftrs, 50) # 50 species


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, images_per_class=50, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images_per_class = images_per_class
        self.images = sorted(os.listdir(root_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = idx // self.images_per_class
        return image, label

In [ ]:
# Example usage:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the desired size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

In [ ]:
import random

num_train_per_class = 40
num_val_per_class = 10

# List to store indices of images for training and validation
train_indices = []
val_indices = []

# Iterate over classes
for class_idx in range(50):
    # List all image filenames for the current class
    class_images = [f'{class_idx:04d}.jpg' for class_idx in range(class_idx * 50 + 1, (class_idx + 1) * 50 + 1)]
    # Shuffle the image filenames
    random.shuffle(class_images)
    # Assign indices for training and validation sets
    train_indices.extend(class_images[:num_train_per_class])
    val_indices.extend(class_images[num_train_per_class:])

# Create subsets for training and validation
train_dataset = Subset(CustomDataset(root_dir='00_test_img_input/train/images'), train_indices)
val_dataset = Subset(CustomDataset(root_dir='00_test_img_input/train/images'), val_indices)

# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
# train_data = CustomDataset('00_test_img_input/train/images', images_per_class=50, transform=data_transform)



# train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# valid_data = CustomDataset('00_test_img_input/test/images', images_per_class=50, transform=data_transform)
# valid_data_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

In [ ]:
from torch.optim import Adam

In [ ]:
creterion = nn.CrossEntropyLoss()
creterion.to(device)
optimizer = Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from PIL import Image

In [ ]:
num_epochs = 15
best_accuracy = 0
early_stopping_counter = 0
early_stopping_limit = 5  # Adjust as needed

for epoch in range(num_epochs):
  model.train()
  for x, y in tqdm(train_loader):
    # x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = creterion(y_pred, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  model.eval()
  val_predictions = []
  val_targets = []
  with torch.no_grad():
    for x, y in tqdm(val_loader):
      # x, y = x.to(device), y.to(device)
      y_pred = model(x)
      val_predictions.extend(torch.argmax(y_pred, dim=1).cpu().tolist())
      val_targets.extend(y.cpu().y.tolist())

  accuracy = accuracy_score(val_targets, val_predictions)

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    early_stopping_counter = 0
  else:
    early_stopping_counter += 1

  if early_stopping_counter == early_stopping_limit:
    print(f'Early stopping at epoch {epoch} due to lack of improvement in validation accuracy.')
    break

  print(f'Epoch {epoch}: Validation Accuracy: {accuracy}')

In [ ]:
# from torchvision import transforms
# from PIL import Image

# # base transformations
# transform = transforms.Compose([
#     transforms.Grayscale(),
#     transforms.ToTensor(),
# ])


# def predict_number(image_path):
#     image = Image.open(image_path)
#     image = transform(image).unsqueeze(0)

#     with torch.no_grad():
#         output = model(image)

#     _, predicted = torch.max(output, 1)
#     return predicted.item(), image

# image_paths = ["Centered_8.png", "Centered_3.png", "Uncentered_3.png"]

# for image_path in image_paths:
#     predicted_number, image = predict_number(image_path)

#     plt.imshow(image.squeeze(), cmap='gray')
#     plt.title(f"Predicted Number: {predicted_number}")
#     plt.axis('off')
#     plt.show()

# torch.save(model.state_dict(), "model_weights.pth")